# Фреймворк Adversarial Robustness Toolbox

Adversarial Robustness Toolbox (ART) v1.14 - это фреймворк, предназначенный для создания, анализа и обработки атак на модели машинного обучения. В этом ответе мы рассмотрим ключевые аспекты ART, выберем сценарий атаки и реализуем его на платформе Google Colab.

Генерация атак: ART предоставляет различные алгоритмы для генерации атак, такие как FGSM, BIM, PGD и другие.

# Fast Gradient Sign Method (FGSM)

Основная идея FGSM заключается в следующем:

Вычисляется функция потерь модели, которая измеряет, насколько хорошо модель предсказывает правильные метки для входных данных.

Вычисляется градиент функции потерь по отношению к входным данным. Этот градиент показывает направление, в котором нужно изменить входные данные, чтобы увеличить ошибку модели.

Входные данные изменяются в направлении знака градиента на небольшую величину, определяемую параметром 𝜖 ϵ. Это изменение делает входные данные "враждебными" для модели, то есть такими, которые модель будет классифицировать неправильно.


# Проведение атаки с помощью сценария FGSM

In [1]:
# Устанавливаем библиотеку Adversarial Robustness Toolbox (ART) для защиты от атак на нейросети
!pip install adversarial-robustness-toolbox

import torch  # Фреймворк для работы с нейросетями
import torch.nn as nn  # Модуль для создания нейросетевых слоёв
import torch.nn.functional as F  # Функции активации, пулинга и т.д.
import torch.optim as optim  # Оптимизаторы для обучения
import numpy as np  # Работа с массивами
from art.attacks.evasion import FastGradientMethod  # Метод атаки "Быстрый градиентный метод"
from art.estimators.classification import PyTorchClassifier  # Обёртка для PyTorch-модели
from art.utils import load_mnist  # Загрузка датасета MNIST

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:00


# Конвертация данных в формат PyTorch

In [2]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()  # Загружаем данные MNIST (рукописные цифры)
x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32) # Транспонируем данные из формата (N, H, W, C) в (N, C, H, W), как требует PyTorch
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)# Преобразуем в float32

# Создание модели

In [3]:
# Определяем архитектуру простой свёрточной нейросети (CNN)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)# Первый свёрточный слой: 1 канал на вход, 32 на выход, ядро 3x3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) # Второй свёрточный слой: 32 канала на вход, 64 на выход
        self.fc1 = nn.Linear(64 * 7 * 7, 128) # Полносвязный слой: вход 64*7*7 (после пулинга), выход 128
        self.fc2 = nn.Linear(128, 10)# Выходной слой: 128 -> 10 (по числу классов цифр 0-9)

    def forward(self, x):
        x = F.relu(self.conv1(x))# Проход через первый свёрточный слой + ReLU + макс-пулинг (уменьшение в 2 раза)
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))# Проход через второй свёрточный слой + ReLU + макс-пулинг
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7) # "Вытягиваем" тензор в вектор для полносвязных слоёв
        x = F.relu(self.fc1(x))# Полносвязные слои с активацией ReLU
        x = self.fc2(x)
        return x

model = SimpleCNN() # Создаём экземпляр модели
criterion = nn.CrossEntropyLoss() # Функция потерь - кросс-энтропия
optimizer = optim.Adam(model.parameters(), lr=0.001) # Оптимизатор

# Создание классификатора для ART

In [4]:
# Обёртка ART для PyTorch-модели (чтобы использовать атаки и защиты)
classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),  # Диапазон значений пикселей
    loss=criterion,  # Функция потерь
    optimizer=optimizer,  # Оптимизатор
    input_shape=(1, 28, 28),  # Формат входных данных (1 канал, 28x28)
    nb_classes=10,  # Число классов (10 цифр)
)

# Создание атаки с помощью FGSM

In [5]:
# Создаём атаку "Быстрый градиентный метод" (FGSM) с параметром eps=0.2
attack = FastGradientMethod(estimator=classifier, eps=0.2)
# Генерируем "вредные" примеры на основе тестовых данных
x_test_adv = attack.generate(x=x_test)

In [6]:
predictions = classifier.predict(x_test_adv) # Проверяем точность модели на атакованных данных
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test) # Сравниваем предсказания с истинными метками
print("Точность: {:.2f}%".format(accuracy * 100))

Точность: 9.88%


# Обучение модели

In [7]:
from art.defences.trainer import AdversarialTrainer

adv_trainer = AdversarialTrainer(classifier, attack)# Создаём с защитой от атак

nb_epochs = 5 # Дообучаем модель на тренировочных данных (5 эпох)
adv_trainer.fit(x_train, y_train, nb_epochs=nb_epochs)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/5 [00:00<?, ?it/s]

# Оценка модели после проведенного обучения

In [8]:
predictions_adv_trained = adv_trainer.predict(x_test_adv) # Проверяем точность на атакованных данных после защиты
accuracy_adv_trained = np.sum(np.argmax(predictions_adv_trained, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Точность на атакованных данных после обучения: {:.2f}%".format(accuracy_adv_trained * 100))

predictions_clean_trained = adv_trainer.predict(x_test) # Проверяем точность на обычных данных после защиты
accuracy_clean_trained = np.sum(np.argmax(predictions_clean_trained, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Точность на неатакованных данных после обучения: {:.2f}%".format(accuracy_clean_trained * 100))

Точность на атакованных данных после обучения: 97.97%
Точность на неатакованных данных после обучения: 98.67%
